In [41]:
# ruff: noqa: F401, E402, T201, T203, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pformat, pprint
from typing import Callable, Iterable, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from IPython.display import HTML, clear_output, display

import arrays
import combinatorics
import graphs
import grid
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import stack
import strings
import trees

for m in (
    arrays,
    combinatorics,
    graphs,
    grid,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

np.set_printoptions(linewidth=110)
print()


In [279]:
(1, 2, 3, 4) > (1, 0)


True

In [123]:
def shortest_path_with_walls(grid: list, k: int) -> int:
    n, m = len(grid), len(grid[0])
    v = [[None] * m for _ in grid]

    i = 0
    q = [(1, 0, 0, 0)]
    while q:
        i += 1
        if i % 2 == 1:
            display(v, clear=True)
            display(len(q))
        ln, walls, x, y = heappop(q)
        if x == n - 1 and y == m - 1:
            return ln - 1
        if v[x][y] and v[x][y] < (ln, walls):
            continue
        v[x][y] = (ln, walls)
        for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
            cx, cy = x + dx, y + dy
            if not (0 <= cx < n and 0 <= cy < m):
                continue
            cwalls = walls + grid[cx][cy]
            if v[cx][cy] and v[cx][cy] < (ln + 1, cwalls):
                continue
            if cwalls > k:
                continue
            v[cx][cy] = (ln + 1, cwalls)
            heappush(q, (ln + 1, cwalls, cx, cy))
    return -1


In [139]:
def shortest_path_with_walls(grid: list, k: int) -> int:
    n, m = len(grid), len(grid[0])
    v = [[None] * m for _ in grid]

    i = 0
    q = [(1, 0, 0, 0)]
    while q:
        i += 1
        if i % 2 == 1:
            display(v, clear=True)
            display(len(q))
        ln, walls, x, y = heappop(q)
        if x == n - 1 and y == m - 1:
            return ln - 1
        if v[x][y] and v[x][y] < (ln, walls):
            continue
        v[x][y] = (ln, walls)
        for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
            cx, cy = x + dx, y + dy
            if not (0 <= cx < n and 0 <= cy < m):
                continue
            cwalls = walls + grid[cx][cy]
            if v[cx][cy]:  # and v[cx][cy] < (ln + 1, cwalls):
                continue
            if cwalls > k:
                continue
            v[cx][cy] = (ln + 1, cwalls)
            heappush(q, (ln + 1, cwalls, cx, cy))
    return -1


In [253]:
def shortest_path_with_walls(g: list, k: int) -> int:
    n, m = len(g), len(g[0])
    mn = m * n
    v = [[mn] * m for _ in g]
    q = [(0, g[0][0], 0, 0)]
    i = 0
    while q:
        i += 1
        if i % 2 == 1:
            display(v, clear=True)
            display(len(q))
        ln, w, x, y = heappop(q)
        if x == n - 1 and y == m - 1:
            display(v, clear=True)
            display(len(q))
            return ln
        if v[x][y] < ln:
            continue
        v[x][y] = ln
        ln += 1
        for cx, cy in ((x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)):
            if 0 <= cx < n and 0 <= cy < m and v[cx][cy] > ln and w + g[cx][cy] <= k:
                v[cx][cy] = ln
                heappush(q, (ln, w + g[cx][cy], cx, cy))
    return -1


In [273]:
def shortest_path_with_walls(g: list, k: int) -> int:
    n, m = len(g), len(g[0])
    mn = m * n
    v = [[(mn, 0, 0, 0)] * m for _ in g]
    q = deque([(0, g[0][0], 0, 0)])
    i = 0
    while q:
        i += 1
        if i % 10 == 1:
            display(v, clear=True)
            display(len(q))
        pv = ln, w, px, py = q.popleft()
        if px == n - 1 and py == m - 1:
            display(v, clear=True)
            display(len(q))
            return ln
        if pv > v[px][py]:
            continue
        v[px][py] = pv
        for x, y in ((px - 1, py), (px + 1, py), (px, py - 1), (px, py + 1)):
            if 0 <= x < n and 0 <= y < m:
                nv = (ln + 1, w + g[x][y], x, y)
                if v[x][y] > nv and nv[1] <= k:
                    v[x][y] = nv
                    q.append(nv)
    return -1


In [261]:
g = [
    [0, 1, 1, 1, 1],
    [0, 1, 0, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 0, 1, 0],
    [1, 1, 1, 1, 0],
]
shortest_path_with_walls(g, 2)


[[0, 1, 2, 25, 25],
 [1, 2, 3, 4, 5],
 [2, 3, 4, 5, 6],
 [3, 4, 5, 6, 7],
 [4, 5, 6, 25, 8]]

0

8

In [272]:
g = grid.parse(
    """\
    0 0 1
    1 0 0
    0 0 0
    0 0 0
    1 1 0
    0 0 0
    0 0 0
    0 1 0
    0 0 0
    0 0 1
    0 0 0
    0 0 0
    1 0 0
    0 0 0
    0 0 0
    0 0 0
    0 0 1
    1 0 0
    0 0 0
    0 0 0
    0 0 0
    0 0 0
    0 0 0
    0 0 0
    1 0 0
    0 0 1
    0 0 0
    0 0 0
    0 0 1
    0 1 0
    1 0 0
    0 0 0
    0 0 0""",
)

shortest_path_with_walls(g, 4)


[[(0, 0, 0, 0), (1, 0, 0, 1), (2, 1, 0, 2)],
 [(1, 1, 1, 0), (2, 0, 1, 1), (3, 0, 1, 2)],
 [(2, 1, 2, 0), (3, 0, 2, 1), (4, 0, 2, 2)],
 [(3, 1, 3, 0), (4, 0, 3, 1), (5, 0, 3, 2)],
 [(4, 2, 4, 0), (5, 1, 4, 1), (6, 0, 4, 2)],
 [(5, 2, 5, 0), (6, 1, 5, 1), (7, 0, 5, 2)],
 [(6, 2, 6, 0), (7, 1, 6, 1), (8, 0, 6, 2)],
 [(7, 2, 7, 0), (8, 2, 7, 1), (9, 0, 7, 2)],
 [(8, 2, 8, 0), (9, 2, 8, 1), (10, 0, 8, 2)],
 [(9, 2, 9, 0), (10, 2, 9, 1), (11, 1, 9, 2)],
 [(10, 2, 10, 0), (11, 2, 10, 1), (12, 1, 10, 2)],
 [(11, 2, 11, 0), (12, 2, 11, 1), (13, 1, 11, 2)],
 [(12, 3, 12, 0), (13, 2, 12, 1), (14, 1, 12, 2)],
 [(13, 3, 13, 0), (14, 2, 13, 1), (15, 1, 13, 2)],
 [(14, 3, 14, 0), (15, 2, 14, 1), (16, 1, 14, 2)],
 [(15, 3, 15, 0), (16, 2, 15, 1), (17, 1, 15, 2)],
 [(16, 3, 16, 0), (17, 2, 16, 1), (18, 2, 16, 2)],
 [(17, 4, 17, 0), (18, 2, 17, 1), (19, 2, 17, 2)],
 [(18, 4, 18, 0), (19, 2, 18, 1), (20, 2, 18, 2)],
 [(19, 4, 19, 0), (20, 2, 19, 1), (21, 2, 19, 2)],
 [(20, 4, 20, 0), (21, 2, 20, 1), (22

0

34

In [274]:
g = grid.parse(
    """
0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0
0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1
0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0
0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0
0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0
0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0
0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1 0
0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0
0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1
0 1 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0
0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0""",
)


In [276]:
shortest_path_with_walls(g, 0)


[[(0, 0, 0, 0),
  (1, 0, 0, 1),
  (2, 0, 0, 2),
  (3, 0, 0, 3),
  (4, 0, 0, 4),
  (5, 0, 0, 5),
  (6, 0, 0, 6),
  (7, 0, 0, 7),
  (8, 0, 0, 8),
  (9, 0, 0, 9),
  (10, 0, 0, 10),
  (510, 0, 0, 0),
  (510, 0, 0, 0),
  (17, 0, 0, 13),
  (510, 0, 0, 0),
  (19, 0, 0, 15),
  (20, 0, 0, 16)],
 [(1, 0, 1, 0),
  (2, 0, 1, 1),
  (3, 0, 1, 2),
  (4, 0, 1, 3),
  (5, 0, 1, 4),
  (6, 0, 1, 5),
  (7, 0, 1, 6),
  (8, 0, 1, 7),
  (9, 0, 1, 8),
  (10, 0, 1, 9),
  (11, 0, 1, 10),
  (12, 0, 1, 11),
  (510, 0, 0, 0),
  (16, 0, 1, 13),
  (17, 0, 1, 14),
  (18, 0, 1, 15),
  (510, 0, 0, 0)],
 [(2, 0, 2, 0),
  (3, 0, 2, 1),
  (510, 0, 0, 0),
  (5, 0, 2, 3),
  (510, 0, 0, 0),
  (7, 0, 2, 5),
  (8, 0, 2, 6),
  (510, 0, 0, 0),
  (10, 0, 2, 8),
  (11, 0, 2, 9),
  (12, 0, 2, 10),
  (13, 0, 2, 11),
  (14, 0, 2, 12),
  (15, 0, 2, 13),
  (510, 0, 0, 0),
  (19, 0, 2, 15),
  (510, 0, 0, 0)],
 [(3, 0, 3, 0),
  (4, 0, 3, 1),
  (510, 0, 0, 0),
  (6, 0, 3, 3),
  (7, 0, 3, 4),
  (8, 0, 3, 5),
  (9, 0, 3, 6),
  (510, 0, 0, 0)

0

45